In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

In [8]:
def max_accuracy(accuracies, diagonal = True):
    """
    Return the maximum accuracy for each session.
    """
    # average over folds
    avg = np.mean(accuracies, axis=1)

    acc_max = []
    for i in range(avg.shape[0]): # looping over sessions
        if diagonal:
            max_acc = np.max(avg[i].diagonal()*100)
        else:
            max_acc = np.max(avg[i]*100)
        acc_max.append(max_acc)
    
    return acc_max

## Within session decoding

### Difference in accuracy between the two strategies

In [9]:
lbo_file  = 'accuracies_LDA_lbo'
lbo = np.load(f'accuracies/{lbo_file}.npy', allow_pickle=True)


bs_file = 'accuracies_LDA_prop'
bs = np.load(f'accuracies/{bs_file}.npy', allow_pickle=True)

lbo_acc = max_accuracy(lbo, diagonal = True)
bs_acc = max_accuracy(bs, diagonal = True)


# do a t-test
print(stats.ttest_rel(lbo_acc, bs_acc))

# print mean and std
print(f'LBO: {np.mean(lbo_acc):.2f} +/- {np.std(lbo_acc):.2f}')
print(f'BS: {np.mean(bs_acc):.2f} +/- {np.std(bs_acc):.2f}')

Ttest_relResult(statistic=1.4806748344263212, pvalue=0.18919355427264875)
LBO: 66.40 +/- 2.11
BS: 65.93 +/- 2.54


## Cross decoding

In [10]:
file_source = 'cross_decoding_ncv_5'
cross_source = np.load(f'accuracies/{file_source}.npy')

file_sens = 'cross_decoding_sens_ncv_5'
cross_sens = np.load(f'accuracies/{file_sens}.npy')

# list of 7 empty lists
accuracies_source = []
accuracies_sens = []

for i in range(7):
    tmp_source = cross_source[i, :, :, :]
    tmp_sens = cross_sens[i, :, :, :]

    # remove the within session accuracies
    tmp_source = np.delete(tmp_source, i, axis=0)
    tmp_sens = np.delete(tmp_sens, i, axis=0)
    for j in range(tmp_source.shape[0]):
        accuracies_source.append(np.max(tmp_source[j, :, :])*100)
        accuracies_sens.append(np.max(tmp_sens[j, :, :])*100)

In [11]:
# do a t-test
print(stats.ttest_rel(accuracies_source, accuracies_sens))

# print mean and std
print(f'Source: {np.mean(accuracies_source):.2f} +/- {np.std(accuracies_source):.2f}')
print(f'Sens: {np.mean(accuracies_sens):.2f} +/- {np.std(accuracies_sens):.2f}')

Ttest_relResult(statistic=-2.623778764830078, pvalue=0.01215826551731701)
Source: 61.60 +/- 1.50
Sens: 62.40 +/- 1.62


# playground aka mess!!!

In [ ]:
props = np.load('./accuracies/accuracies_LDA_props.npy', allow_pickle=True).squeeze() # proportional session

for i in range(len(props)):
    plt.imshow(props[i], cmap='viridis')
    plt.show()

In [ ]:
# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)
# standard deviation within session
std_sessions = np.load('../ERF_analysis/std_sessions.npy')
# average over timepoints
std_sessions = np.mean(std_sessions, axis=1)

std_test = std_sessions.tolist()*7
std_train = [std_sessions[i] for i in range(7) for j in range(7)]


# create the design matrix
design = pd.DataFrame({'std_test': std_test, 'std_train': std_train, 'accuracy': accuracies})

y, X = dmatrices('accuracy ~ std_train + std_test + std_train:std_test', design)
#y, X = dmatrices('accuracy ~ strategy:std', design)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()